This script can be used to predict the pixel-wise density of the PS points for a given region. 

After the variables specifying the region of interest are set, the code downloads and pre-processes the input data:
- the osm dataset that's used to create an infrastructure map
- the long-term coherence.

Then, it applies a pre-trained model (downloaded automatically from HuggingFace) to this inputs and generates prediction of PS density that are saved as a geotiff file.

Note: the current version only works for regions of interest within one-by-one degree tile. Future updates will extend the functionality to remove this constraint. 

# Import libraries

In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../..")))
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../../..")))
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "src")))

In [2]:
import os
import warnings
import time
import math

from huggingface_hub import hf_hub_download

import torch
from torch.utils.data import DataLoader

from torchgeo.datasets import (
    stack_samples,
)
from torchgeo.samplers import GridGeoSampler

from lightning.pytorch import seed_everything

In [3]:
from utils.find_repo import find_repo_root

from ps_predictions.utils.data_download import calculate_area, download_osm

from ps_predictions.utils.torchgeo_classes_def import PixelwiseRegressionTask

from ps_predictions.utils.torchgeo_fun_to_read_input import (
    scale_coh,
    RasterDataset_imgs,
)
from ps_predictions.utils.prediction_plots import (
    georreferenced_chip_generator,
    merge_georeferenced_chips,
)

from ps_predictions.utils.infrastructure_map_generation import generate_infrastructure_map


In [4]:
# Check if torch with cuda enabled and working
# torch.zeros(1).cuda()
torch.cuda.is_available()

True

In [5]:
# Ignore warning about number of warnings
# See https://github.com/Lightning-AI/lightning/issues/10182
warnings.filterwarnings("ignore", ".*does not have many workers.*")

# Set-up region to be analysed

In [6]:
# !!! FOR NOW IT ONLY WORKS IF THE REGION IS WITHIN ONE BY ONE DEGREE TILE (TO BE UPDATED IN FUTURE VERSIONS) !!!
x = [18.53, 18.9]
y = [50.05, 50.5]

In [7]:
print(f"Area: {calculate_area(x,y):.2f} km²")

Area: 1321.90 km²


# Set-up variables

In [ ]:
# Define where all data should be saved

# Find the repo root by looking for a marker file (e.g., .git)
repo_root = find_repo_root()

# Define the path to the data
root = os.path.join(repo_root, "data")
folder_name = "ps_prediction_with_ML_model"
experiment_name = "experiment_slaskie"

path_prefix = os.path.join(root, folder_name)
experiment_dir = os.path.join(path_prefix, experiment_name)

# Path for saving results (such as model checkpoints)
os.makedirs(experiment_dir, exist_ok=True)


In [9]:
# Define specific OSM file to use (depending on the region of interest)
# The more specific, i.e. the smaller the file, the faster the code will run
filename_OSM = "europe/poland/slaskie-latest.osm.pbf"
osm_pbf_location = os.path.join(experiment_dir, "slaskie-latest.osm.pbf")

# Download and pre-process inputs

In [10]:
# Set variables required for infrastructure map generation and coherence download
tags_high = {"highway": ["motorway", "trunk", "primary", "secondary"]}
tags_build = {"building": ["yes"]}
tags_rail = {"railway": ["rail"]}

min_x = min(x)
min_y = min(y)
max_x = max(x)
max_y = max(y)

# Convert min_x and min_y to integers for use in formatting the latitude and longitude
west = math.floor(min_x)
south = math.floor(min_y)
east = math.ceil(max_x)
north = math.ceil(max_y)

# Convert min_x and min_y to integers for use in formatting the latitude and longitude
lon = int(min_x)
lat = int(min_y)

# Format the latitude and determine the season based on whether the latitude is positive or negative
if south >= 0:
    lat_f = "N{:02d}".format(south + 1)
    season = "summer"
else:
    lat_f = "S{:02d}".format(abs(south + 1))
    season = "winter"
    
# Format the longitude based on whether it is positive or negative
if west >= 0:
    lon_f = "E{:03d}".format(west)
else:
    lon_f = "W{:03d}".format(abs(west))
    
# Print the currently processed tile's coordinates
print(
    "Currently processed tile lat:{},{} lon:{},{}".format(min_y, max_y, min_x, max_x),
    flush=True,
)

# Define bounds for the prediction tile
bounds = (min_x, min_y, max_x, max_y)
bounds_name = (int(min_x), int(min_y), int(max_x), int(max_y))
bounds_ltcoh = (
    math.floor(min_x),
    math.floor(min_y),
    math.ceil(max_x),
    math.ceil(max_y),
)

Currently processed tile lat:50.05,50.5 lon:18.53,18.9


## Download long-term coherence

In [11]:
# Start of long-term coherence section
start_time = time.time()

# Define the path for the long-term coherence tile
coh_tile_path = "{coh_dir}/{lat}{lon}/coh/{lat}{lon}_{season}_vv_rho.tif".format(
    lat=lat_f, lon=lon_f, season=season, coh_dir=experiment_dir
)

# Check if the long-term coherence tile already exists
# If not, try to download it
if os.path.exists(coh_tile_path):
    print(
        "Long-term coherence tile  {}{} is already downloaded.".format(lat_f, lon_f),
        flush=True,
    )
else:
    print(
        "\n Long-term coherence tile {}{} is missing! Trying to download \n".format(
            lat_f, lon_f
        ),
        flush=True,
    )

    try:
        os.system(
            f"aws s3 cp s3://sentinel-1-global-coherence-earthbigdata/data/tiles/{lat_f}{lon_f}/"
            f"{lat_f}{lon_f}_{season}_vv_rho.tif --no-sign-request "
            f"{experiment_dir}/{lat_f}{lon_f}/coh/{lat_f}{lon_f}_{season}_vv_rho.tif"
        )

        if os.path.exists(
            "{coh_dir}/{lat}{lon}/coh/{lat}{lon}_{season}_vv_rho.tif".format(
                lat=lat_f, lon=lon_f, season=season, coh_dir=experiment_dir
            )
        ):
            print("Download of {}{} tiles ended.".format(lat_f, lon_f))
        else:
            print("\n Download of tile {}{} failed. \n".format(lat_f, lon_f))

    except BaseException as error:
        print(
            "There is a problem with the download of the file. Please see following line to identify the error."
        )
        print("An exception occurred: {}".format(error))

    except Exception as e:
        print("An exception occurred: {}".format(e))
        
# Calculate the total execution time for checking the long-term coherence file
end_time = time.time()
total_time = end_time - start_time
print(
    f"Total execution time for downloading the long term cohrence file: {total_time:.2f} seconds",
    flush=True,
)


 Long-term coherence tile N51E018 is missing! Trying to download 



download: s3://sentinel-1-global-coherence-earthbigdata/data/tiles/N51E018/N51E018_summer_vv_rho.tif to ../../data/testing_code_for_publishing3/experiment2_slaskie/N51E018/coh/N51E018_summer_vv_rho.tif
Download of N51E018 tiles ended.
Total execution time for downloading the long term cohrence file: 17.84 seconds


## Generate infrastructure map

In [12]:
# Download required OSM data
download_osm(filename_OSM, osm_pbf_location)

Progress: 100.0%
Download complete! File saved as /mnt/c/Users/Dominika/Desktop/Code_for_AWS_data/bridge-risk-assessment-insar/data/testing_code_for_publishing3/experiment2_slaskie/slaskie-latest.osm.pbf


In [13]:
# Define the directory for the infrastructure map raster
infrmap_raster_dir = os.path.join(
    experiment_dir,
    "{}{}".format(lat_f, lon_f),
    "infr_map",
    "{}{}_infrs_map.tif".format(lat_f, lon_f),
)

In [14]:
# Check if the infrastructure map already exists
if os.path.exists(infrmap_raster_dir):
    print(
        "Infrastructure map {}{} is already generated.".format(lat_f, lon_f),
        flush=True,
    )
# If the infrastructure map does not exist, start the generation process
else:
    generate_infrastructure_map(
        lat_f,
        lon_f,
        min_x,
        min_y,
        max_x,
        max_y,
        experiment_dir,
        osm_pbf_location,
        infrmap_raster_dir,
        osm_convert_path = "/usr/bin/osmconvert"
    )

/mnt/c/Users/Dominika/Desktop/Code_for_AWS_data/bridge-risk-assessment-insar/data/testing_code_for_publishing3/experiment2_slaskie/N51E018/N51E018.o5m started!


/mnt/c/Users/Dominika/Desktop/Code_for_AWS_data/bridge-risk-assessment-insar/data/testing_code_for_publishing3/experiment2_slaskie/N51E018/N51E018.o5m finished!
Converting of o5m for N51E018 is done
1637830 20277 12414
Time for df creation: 131.27 seconds
Time for raster generation: 230.43 seconds


# Download and set-up the model

## Download model

In [15]:
model_dir = os.path.join(path_prefix, "best_models_15122023")
os.makedirs(model_dir, exist_ok=True)

# Download the specific checkpoint file
checkpoint_path = hf_hub_download(
    repo_id="dominika-malinowska/ps_predictions",
    filename="seed_54321_20231214-203338.ckpt",
    local_dir=model_dir  # download to this folder
)

print(f"Checkpoint downloaded to: {checkpoint_path}")

seed_54321_20231214-203338.ckpt:   0%|          | 0.00/293M [00:00<?, ?B/s]

Checkpoint downloaded to: /mnt/c/Users/Dominika/Desktop/Code_for_AWS_data/bridge-risk-assessment-insar/data/testing_code_for_publishing3/best_models_15122023/seed_54321_20231214-203338.ckpt


## Set-up model parameters 

In [16]:
# Constant parameters
# Those shouldn't be changed if you want to use the model in the same way as in the paper
in_channels = 2  # Number of channels in input image
ignore_index = None  # Optional integer class index to ignore in the loss and metrics
lr_patience = 10  # Patience for learning rate scheduler
min_delta_lr = 0.01
patience = 15  # Patience for early stopping
min_delta = (
    0.01  # when change in val_loss less than that for appropriate number of epochs
)
# early stopping will be triggered
max_epochs = 300  # maximum number of epochs
param_opts = [
    [16, 64, None, "resnet34", None, "mae", 0.00776601, 0.9, 0.99, 0.01],
]
patch_size = param_opts[0][1]
batch_size = param_opts[0][0]
# Select overlap parameter for merging patches in prediction
overlap = 8

# GPU settings
gpu_id = 0
device = torch.device(f"cuda:{gpu_id}")
num_dataloader_workers = 0

param_const = [
    in_channels,
    ignore_index,
    lr_patience,
    min_delta_lr,
    patience,
    min_delta,
    max_epochs,
    gpu_id,
    device,
    num_dataloader_workers,
]

In [17]:
# Define the path for the PS prediction raster
file_name = os.path.join(
    experiment_dir, "{}{}".format(lat_f, lon_f), "{}{}_PS_pred.tif".format(lat_f, lon_f)
)

# Create the directory for storing outputs if it doesn't exist
dir_coord = os.path.join(experiment_dir, "{}{}".format(lat_f, lon_f))
if not os.path.exists(dir_coord):
    os.makedirs(dir_coord)

## Read trained model

In [18]:
# Read trained model
model = PixelwiseRegressionTask.load_from_checkpoint(
    checkpoint_path=os.path.join(
        model_dir, "seed_54321_20231214-203338.ckpt"
    )
)

In [19]:
# Set-up seed
seed = 54321
seed_everything(seed, workers=True)
generator = torch.Generator().manual_seed(seed)

Seed set to 54321


# Prepare input data for the model

In [20]:
# Create RasterDataset objects for coherence and infrastructure map images
imgs_coh = RasterDataset_imgs(
    paths=os.path.join(
        experiment_dir,
        "{}{}".format(lat_f, lon_f),
        "coh",
    ),
    transforms=scale_coh,
)
imgs_infrmap = RasterDataset_imgs(
    paths=os.path.join(experiment_dir, "{}{}".format(lat_f, lon_f), "infr_map")
)
# Combine the two RasterDataset objects
imgs_input = imgs_coh & imgs_infrmap

Converting RasterDataset_imgs res from 0.0008333333333333373 to 0.00083333333


In [21]:
# Set the dataset to predict
dataset_to_pred = imgs_input

In [22]:
# Create a GridGeoSampler object for the dataset
sampler_pred = GridGeoSampler(dataset_to_pred, patch_size, patch_size - overlap)

In [23]:
# Create dataloader
dataloader_interf = DataLoader(
    dataset=dataset_to_pred,
    batch_size=batch_size,
    sampler=sampler_pred,
    num_workers=0,
    collate_fn=stack_samples,
    generator=generator,
)

# Run predictions

In [24]:
model.eval()

# Generate geotiff
# Get the pixel size and CRS of the dataset
pixel_size = dataset_to_pred.res
crs = dataset_to_pred.crs.to_epsg()


# Generate prediction chips
start = time.time()  # Start measuring the time
chips_generator = georreferenced_chip_generator(
    dataloader_interf, model, crs, pixel_size
)
print("The time taken to predict was: ", time.time() - start, flush=True)

# Save the prediction chips as a geotiff
start = time.time()  # Start measuring the time
merge_georeferenced_chips(chips_generator, file_name, overlap, bounds)
print(
    "The time taken to generate a georrefenced image and save it was: ",
    time.time() - start,
    flush=True,
)

The time taken to predict was:  7.7916483879089355
The time taken to generate a georrefenced image and save it was:  0.10680079460144043
